In [1]:
import pandas as pd
import numpy as np
from untrade.client import Client


def process_data(df):
   """
   Process BTC data to generate trading signals based on different strategies.
   Supported strategies: 'bollinger', 'macd', 'moving_average_crossover', 'custom_macd'.


   Parameters:
   df (pd.DataFrame): DataFrame with columns 'open', 'high', 'low', 'close'
   strategy (str): The trading strategy to apply.


   Returns:
   pd.DataFrame: DataFrame with added columns for indicators and trading signals
   """
   # Make a copy of the dataframe
   df = df.copy()


   # Calculate common indicators
   df['RSI'] = calculate_rsi(df['close'], period=14)
   df['ATR'] = calculate_atr(df)
   df['EMA_6'] = df['close'].ewm(span=6, adjust=False).mean()


   # Initialize columns
   df['Signal'] = 0
   df['trade'] = 0
   df['SL'] = np.nan
   df['TP'] = np.nan
   df['trade_type'] = "square-off"


       # MACD Calculation
   df['EMA_12'] = df['close'].ewm(span=5, adjust=False).mean()
   df['EMA_26'] = df['close'].ewm(span=13, adjust=False).mean()
   df['MACD'] = df['EMA_12'] - df['EMA_26']
   df['Signal_Line'] = df['MACD'].ewm(span=5, adjust=False).mean()


   for i in range(1, len(df)):
       # Bullish Signal: MACD Fast crosses above Slow and price is above EMA_6
       if (
           df['MACD'].iloc[i] > df['Signal_Line'].iloc[i]
           and df['MACD'].iloc[i - 1] <= df['Signal_Line'].iloc[i - 1]
           and df['close'].iloc[i] > df['EMA_6'].iloc[i]
       ):
           df.loc[df.index[i], 'Signal'] = 1
           df.loc[df.index[i], 'trade'] = 1
           df.loc[df.index[i], 'trade_type'] = "long"
           df.loc[df.index[i], 'SL'] = df['close'].iloc[i] - 1.5 * df['ATR'].iloc[i]
           df.loc[df.index[i], 'TP'] = df['close'].iloc[i] + 3 * df['ATR'].iloc[i]


       # Bearish Signal: MACD Fast crosses below Slow and price is below EMA_6
       elif (
           df['MACD'].iloc[i] < df['Signal_Line'].iloc[i]
           and df['MACD'].iloc[i - 1] >= df['Signal_Line'].iloc[i - 1]
           and df['close'].iloc[i] < df['EMA_6'].iloc[i]
       ):
           df.loc[df.index[i], 'Signal'] = -1
           df.loc[df.index[i], 'trade'] = -1
           df.loc[df.index[i], 'trade_type'] = "short"
           df.loc[df.index[i], 'SL'] = df['close'].iloc[i] + 1.5 * df['ATR'].iloc[i]
           df.loc[df.index[i], 'TP'] = df['close'].iloc[i] - 3 * df['ATR'].iloc[i]


   return df






def calculate_rsi(series, period=14):
   """Helper function to calculate RSI."""
   delta = series.diff(1)
   gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
   loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
   rs = gain / loss
   return 100 - (100 / (1 + rs))




def calculate_atr(df, period=14):
   """Helper function to calculate ATR."""
   df['TR'] = pd.DataFrame({
       'hl': df['high'] - df['low'],
       'hc': abs(df['high'] - df['close'].shift(1)),
       'lc': abs(df['low'] - df['close'].shift(1))
   }).max(axis=1)
   return df['TR'].rolling(window=period).mean()


def strat(data):
   """
   Apply strategy to filter consecutive signals.


   Parameters:
   - data (DataFrame): Dataframe containing buy/sell signals.


   Returns:
   - data (DataFrame): Dataframe with filtered signals.
   """
   signal = []
   prev = None
   for value in data["Signal"]:
       if value == prev:
           signal.append(0)
       else:
           signal.append(value)
       prev = value


   data["signals"] = signal


   # Keep only the required columns
   data = data[['datetime', 'open', 'high', 'low',
            'close', 'volume', 'signals', 'trade_type']]


   return data


def perform_backtest(csv_file_path):
   """
   Perform backtesting using the untrade SDK.


   Parameters:
   - csv_file_path (str): Path to the CSV file containing historical price data and signals.


   Returns:
   - result (generator): Generator object that yields backtest results.
   """
   # Create an instance of the untrade client
   client = Client()


   # Perform backtest using the provided CSV file path
   result = client.backtest(
       jupyter_id="vraj2811",  # your Jupyter ID
       file_path=csv_file_path,
       leverage=1,  # Adjust leverage as needed
   )


   return result




# Load ETH data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)


# Save processed data to CSV file
res = strat(res)
res.to_csv("processed_data.csv", index=False)


# Perform backtest on processed data
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)


# Print the backtest result
for value in backtest_result:
   print(value)

SSLError: HTTPSConnectionPool(host='api.untrade.io', port=443): Max retries exceeded with url: /v1/untrade/backtest-stream (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

In [2]:
print(data.head())

   Unnamed: 0             datetime    open    high     low   close     volume
0           0  2019-12-01 00:00:00  151.38  151.38  145.70  147.09  69972.978
1           1  2019-12-01 04:00:00  147.05  148.64  146.39  146.79  62459.238
2           2  2019-12-01 08:00:00  146.77  148.28  145.50  147.20  68504.046
3           3  2019-12-01 12:00:00  147.21  152.50  147.21  149.41  65606.928
4           4  2019-12-01 16:00:00  149.49  150.55  148.98  150.06  59505.894


In [5]:
import pandas as pd
import numpy as np
from untrade.client import Client


def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                 'close', 'volume', 'signals', 'trade_type']]

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=3,  # Set leverage for backtest
    )

    return result


def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL), take-profit (TP), and trailing stop logic.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    # Create a copy of the dataframe to avoid modifying the original
    df = df.copy()

    # Calculate Exponential Moving Average (EMA) for 30-minute timeframe
    ema_periods = 89 * 48  # 4272 thirty-minute periods
    df['EMA'] = df['close'].ewm(span=ema_periods, adjust=False).mean()

    # Calculate True Range (TR) and Average True Range (ATR)
    df['TR'] = pd.DataFrame({
        'hl': df['high'] - df['low'],  # High-Low range
        'hc': abs(df['high'] - df['close'].shift(1)),  # High-Previous Close
        'lc': abs(df['low'] - df['close'].shift(1))  # Low-Previous Close
    }).max(axis=1)

    atr_periods = 14 * 48  # 672 thirty-minute periods
    # ATR is rolling mean of TR
    df['ATR'] = df['TR'].rolling(window=atr_periods).mean()

    # Initialize columns for trade logic
    df['Signal'] = 0  # Signal column to store buy (1), sell (-1), or square-off signals
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trailing_stop'] = np.nan  # Trailing stop-loss levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', or 'square-off')

    # Loop through data to calculate signals and update trade states
    for i in range(1, len(df)):
        # Get previous and current values for close, EMA, and ATR
        prev_close = df['close'].iloc[i-1]
        curr_close = df['close'].iloc[i]
        prev_ema = df['EMA'].iloc[i-1]
        curr_ema = df['EMA'].iloc[i]
        curr_atr = df['ATR'].iloc[i]

        # Check active trade
        prev_trade = df['trade'].iloc[i-1]

        # Handle bullish crossover (long signal)
        if prev_close <= prev_ema and curr_close > curr_ema:
            if prev_trade == -1:  # Active short trade, square off
                df.loc[df.index[i], 'Signal'] = 2
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                df.loc[df.index[i], 'Signal'] = 1
                df.loc[df.index[i], 'SL'] = curr_close - (0.5 * curr_atr)
                df.loc[df.index[i], 'TP'] = curr_close + (2 * curr_atr)
                df.loc[df.index[i], 'trailing_stop'] = curr_close - (2 * curr_atr)
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'

        # Handle bearish crossover (short signal)
        elif prev_close >= prev_ema and curr_close < curr_ema:
            if prev_trade == 1:  # Active long trade, square off
                df.loc[df.index[i], 'Signal'] = -2
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                df.loc[df.index[i], 'Signal'] = -1
                df.loc[df.index[i], 'SL'] = curr_close + (0.5 * curr_atr)
                df.loc[df.index[i], 'TP'] = curr_close - (2 * curr_atr)
                df.loc[df.index[i], 'trailing_stop'] = curr_close + (2 * curr_atr)
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'

        else:
            # Update trailing stop-loss for ongoing trades
            if prev_trade != 0:  # Active trade
                df.loc[df.index[i], 'trade'] = prev_trade
                prev_trailing_stop = df['trailing_stop'].iloc[i-1]

                if prev_trade == 1:  # Long position
                    new_trailing_stop = curr_close - (2 * curr_atr)
                    df.loc[df.index[i], 'trailing_stop'] = max(prev_trailing_stop, new_trailing_stop)
                    df.loc[df.index[i], 'SL'] = df.loc[df.index[i], 'trailing_stop']
                    df.loc[df.index[i], 'TP'] = df['TP'].iloc[i-1]

                    if curr_close <= df.loc[df.index[i], 'trailing_stop'] or curr_close >= df['TP'].iloc[i-1]:
                        df.loc[df.index[i], 'trade'] = 0
                        df.loc[df.index[i], 'trade_type'] = 'square-off'

                elif prev_trade == -1:  # Short position
                    new_trailing_stop = curr_close + (2 * curr_atr)
                    df.loc[df.index[i], 'trailing_stop'] = min(prev_trailing_stop, new_trailing_stop)
                    df.loc[df.index[i], 'SL'] = df.loc[df.index[i], 'trailing_stop']
                    df.loc[df.index[i], 'TP'] = df['TP'].iloc[i-1]

                    if curr_close >= df.loc[df.index[i], 'trailing_stop'] or curr_close <= df['TP'].iloc[i-1]:
                        df.loc[df.index[i], 'trade'] = 0
                        df.loc[df.index[i], 'trade_type'] = 'square-off'

    # Include volume if not already present
    if 'volume' not in df.columns:
        df['volume'] = np.nan  # Set to NaN if missing

    # Create 'signals' column as integer type
    df['signals'] = df['Signal'].astype(int)

    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type']]

    return df


# Load historical price data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/BTC/BTC_2019_2023_30m.csv")

# Process the data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 17:30:00",
      "Total Trades": 166,
      "Leverage Applied": 3.0,
      "Winning Trades": 13,
      "Losing Trades": 153,
      "No. of Long Trades": 78,
      "No. of Short Trades": 88,
      "Benchmark Return(%)": 324.585,
      "Benchmark Return(on $1000)": 3245.85,
      "Win Rate": 7.831325,
      "Winning Streak": 2,
      "Losing Streak": 35,
      "Gross Profit": 14813.338369,
      "Net Profit": 14066.338369,
      "Average Profit": 84.736978,
      "Maximum Drawdown(%)": 73.170192,
      "Average Drawdown(%)": 11.856826,
      "Largest Win": 10971.438531,
      "Average Win": 1342.584313,
      "Largest Loss": -110.588876,
      "Average Loss": -22.138939,
      "Maximum Holding Time": "196 days 15:59:59",
      "Average Holding Time": "6 days 14:29:27",
      "Maximum Adverse Excursion": 3.995801,
 